In [ ]:
from utils import import_dataset
import numpy as np
import pandas as pd

In [ ]:
from keras.models import Model,Sequential,save_model
from keras.layers import Input,Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [ ]:
N_Channels=3
Final_length=70
Final_breadth=200

In [ ]:
INPUT_SHAPE=(Final_length,Final_breadth,N_Channels)
data_x,data_y_angle,data_y_throttle=import_dataset()

In [ ]:
data_x,data_y_angle,data_y_throttle=shuffle(data_x,data_y_angle,data_y_throttle)

In [ ]:
split=int(0.85*data_x.shape[0])
X_train=data_x[:split,:,:,:]
Y_train_a=data_y_angle[:split]
Y_train_t=data_y_throttle[:split]
X_test=data_x[split:,:,:,:]
Y_test_a=data_y_angle[split:]
Y_test_t=data_y_throttle[split:]
X_test/=255.0
X_train/=255.0

In [ ]:
inp=Input(shape=INPUT_SHAPE)
c1=Conv2D(filters=128,strides=(2,2),input_shape=INPUT_SHAPE,kernel_size=(3,3),activation="relu")
c2=Conv2D(filters=64,kernel_size=(3,3),activation="relu")
c3=AveragePooling2D()
c4=Conv2D(filters=48,kernel_size=(3,3),activation="relu")
c5=AveragePooling2D()
c6=Conv2D(filters=40,kernel_size=(3,3),activation="relu")
c7=AveragePooling2D()
c8=Dropout(0.6)
c9=Flatten()
c10=Dense(500,activation="relu")
c11=Dense(250,activation="relu")
c12=Dense(100,activation="relu")
c13=Dense(50,activation="relu")
c14=Dense(25,activation="relu")
c15=Dense(10,activation="relu")
c16=Dense(1)
c17=Dense(1)
out1=c16(c15(c14(c13(c12(c11(c10(c9(c8(c7(c6(c5(c4(c3(c2(c1(inp))))))))))))))))
out2=c17(c15(c14(c13(c12(c11(c10(c9(c8(c7(c6(c5(c4(c3(c2(c1(inp))))))))))))))))
model=Model(inputs=[inp],outputs=[out1,out2])
model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=0.0001),loss="mse")

In [ ]:
checkpoint = ModelCheckpoint('modelv6_{val_loss:.4f}.h5',
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=True,
                                 mode='auto')

In [ ]:
model.fit(X_train,[Y_train_a,Y_train_t],batch_size=16,epochs=35,validation_data=[X_test,[Y_test_a,Y_test_t]],callbacks=[checkpoint],shuffle=True)